In [38]:
import requests as r
import json
import xmltodict
import pandas as pd
from dateutil import parser
import matplotlib.pyplot as plt

In [2]:
key = "eQNqoldV2y2KH99ZOgqbQ"
base = "https://www.goodreads.com"

In [3]:
me_r = r.get("{b}/user/show/19575421.xml?key={k}".format(b=base, k=key))
xml_data = me_r.content
me = xmltodict.parse(xml_data)

# print(json.dumps(me, indent=2))
shelves = me["GoodreadsResponse"]["user"]["user_shelves"]
shelf = [x for x in shelves["user_shelf"] if x["name"] == "read"][0]
shelf

OrderedDict([('id',
              OrderedDict([('@type', 'integer'), ('#text', '63015937')])),
             ('name', 'read'),
             ('book_count',
              OrderedDict([('@type', 'integer'), ('#text', '376')])),
             ('exclusive_flag',
              OrderedDict([('@type', 'boolean'), ('#text', 'true')])),
             ('description', OrderedDict([('@nil', 'true')])),
             ('sort', OrderedDict([('@nil', 'true')])),
             ('order', OrderedDict([('@nil', 'true')])),
             ('per_page',
              OrderedDict([('@type', 'integer'), ('@nil', 'true')])),
             ('display_fields', None),
             ('featured',
              OrderedDict([('@type', 'boolean'), ('#text', 'true')])),
             ('recommend_for',
              OrderedDict([('@type', 'boolean'), ('#text', 'false')])),
             ('sticky',
              OrderedDict([('@type', 'boolean'), ('@nil', 'true')]))])

In [25]:
read_shelf = r.get(
    (
        "{b}/review/list/19575421.xml?" "key={k}&v=2" "&shelf=read" "&per_page=200"
    ).format(b=base, k=key)
)
xml_data = read_shelf.content
read_books = xmltodict.parse(xml_data)
rb = read_books["GoodreadsResponse"]["reviews"]["review"]
books = pd.DataFrame(rb)

In [26]:
books.sample(4)

,id,book,rating,votes,spoiler_flag,spoilers_state,shelves,recommended_for,recommended_by,started_at,read_at,date_added,date_updated,read_count,body,comments_count,url,link,owned
95,1655562315,"{'id': {'@type': 'integer', '#text': '39889'},...",5,0,false,none,"{'shelf': {'@name': 'read', '@exclusive': 'tru...",None,None,Wed Jun 01 13:43:07 -0700 2016,Mon Sep 05 08:28:17 -0700 2016,Wed Jun 01 13:43:06 -0700 2016,Mon Sep 05 08:28:17 -0700 2016,1,None,0,https://www.goodreads.com/review/show/1655562315,https://www.goodreads.com/review/show/1655562315,0
29,2499022318,"{'id': {'@type': 'integer', '#text': '37590384...",4,0,false,none,"{'shelf': {'@name': 'read', '@exclusive': 'tru...",None,None,Thu Nov 01 05:35:50 -0700 2018,Mon Nov 12 03:49:37 -0800 2018,Sat Aug 18 23:26:07 -0700 2018,Mon Nov 12 03:49:37 -0800 2018,1,I'm feeling very motivated to ferment things n...,0,https://www.goodreads.com/review/show/2499022318,https://www.goodreads.com/review/show/2499022318,0
197,966088773,"{'id': {'@type': 'integer', '#text': '694246'}...",5,0,false,none,"{'shelf': {'@name': 'read', '@exclusive': 'tru...",None,None,None,None,Fri Jun 13 04:40:26 -0700 2014,Tue Sep 09 09:28:54 -0700 2014,1,None,0,https://www.goodreads.com/review/show/966088773,https://www.goodreads.com/review/show/966088773,0
133,1634696008,"{'id': {'@type': 'integer', '#text': '12659347...",4,0,false,none,"{'shelf': {'@name': 'read', '@exclusive': 'tru...",None,None,None,None,Tue May 10 18:56:07 -0700 2016,Tue May 10 18:56:08 -0700 2016,1,None,0,https://www.goodreads.com/review/show/1634696008,https://www.goodreads.com/review/show/1634696008,0


In [27]:
books.columns

Index(['id', 'book', 'rating', 'votes', 'spoiler_flag', 'spoilers_state',
       'shelves', 'recommended_for', 'recommended_by', 'started_at', 'read_at',
       'date_added', 'date_updated', 'read_count', 'body', 'comments_count',
       'url', 'link', 'owned'],
      dtype='object')

In [28]:
books.drop(['votes', 'spoiler_flag', 'spoilers_state', 'shelves', 'recommended_for', 
            'recommended_by', 'read_count', 'body', 'comments_count','link', 'owned'], 
           axis=1,
           inplace=True)

In [42]:
def parseDateSafe(date):
    if date is not None:
        return parser.parse(date)
    else:
        return parser.parse("Tue Jul 30 00:00:00 -0800 2019")

In [43]:
books["title"] = books.apply(lambda row: row.book["title_without_series"], axis=1)
books["author_1_id"] = books.apply(lambda row: row.book["authors"]["author"]["id"], axis=1)
books["author_1_name"] = books.apply(lambda row: row.book["authors"]["author"]["name"], axis=1)
books["dt_updated"] = books.apply(lambda row: parseDateSafe(row.date_updated), axis=1)
books["dt_started_at"] = books.apply(lambda row: parseDateSafe(row.started_at), axis=1)
books["dt_read_at"] = books.apply(lambda row: parseDateSafe(row.read_at), axis=1)
books["dt_added"] = books.apply(lambda row: parseDateSafe(row.date_added), axis=1)

In [44]:
books.sample(5)

,id,book,rating,started_at,read_at,date_added,date_updated,url,title,author_1_id,author_1_name,dt_updated,dt_started_at,dt_read_at,dt_added
4,2858971757,"{'id': {'@type': 'integer', '#text': '111101'}...",5,Sat Jun 15 07:15:30 -0700 2019,Sun Jul 14 19:04:14 -0700 2019,Sat Jun 15 07:15:30 -0700 2019,Sun Jul 14 19:04:14 -0700 2019,https://www.goodreads.com/review/show/2858971757,Bliss,22595,Peter Carey,2019-07-14 19:04:14-07:00,2019-06-15 07:15:30-07:00,2019-07-14 19:04:14-07:00,2019-06-15 07:15:30-07:00
19,2683040597,"{'id': {'@type': 'integer', '#text': '41814440...",4,Mon Jan 21 01:52:30 -0800 2019,Mon Feb 11 03:36:51 -0800 2019,Mon Jan 21 01:52:30 -0800 2019,Mon Feb 11 03:36:51 -0800 2019,https://www.goodreads.com/review/show/2683040597,"Heroes: Mortals and Monsters, Quests and Adven...",10917,Stephen Fry,2019-02-11 03:36:51-08:00,2019-01-21 01:52:30-08:00,2019-02-11 03:36:51-08:00,2019-01-21 01:52:30-08:00
98,1655559754,"{'id': {'@type': 'integer', '#text': '24968626...",4,None,Sun May 01 00:00:00 -0700 2016,Wed Jun 01 13:40:41 -0700 2016,Thu Jul 20 05:10:04 -0700 2017,https://www.goodreads.com/review/show/1655559754,Crux,160069,Ramez Naam,2017-07-20 05:10:04-07:00,2019-07-30 00:00:00-08:00,2016-05-01 00:00:00-07:00,2016-06-01 13:40:41-07:00
107,1634698956,"{'id': {'@type': 'integer', '#text': '13163263...",4,Sun Jul 16 04:35:29 -0700 2017,Mon Oct 23 14:34:20 -0700 2017,Tue May 10 18:59:04 -0700 2016,Mon Oct 23 14:34:20 -0700 2017,https://www.goodreads.com/review/show/1634698956,"Tempo: Timing, Tactics and Strategy in Narrati...",4762746,Venkatesh G. Rao,2017-10-23 14:34:20-07:00,2017-07-16 04:35:29-07:00,2017-10-23 14:34:20-07:00,2016-05-10 18:59:04-07:00
33,2499022118,"{'id': {'@type': 'integer', '#text': '41019110...",4,Thu Oct 04 03:44:14 -0700 2018,Sun Oct 07 04:51:42 -0700 2018,Sat Aug 18 23:25:43 -0700 2018,Sun Oct 07 04:51:42 -0700 2018,https://www.goodreads.com/review/show/2499022118,It Doesn’t Have to Be Crazy at Work,60619,Jason Fried,2018-10-07 04:51:42-07:00,2018-10-04 03:44:14-07:00,2018-10-07 04:51:42-07:00,2018-08-18 23:25:43-07:00


In [56]:
for index, row in books.iterrows():
    if books.dt_started_at is not None and books.dt_read_at is not None:
        plt.plot_date([books.dt_started_at, books.dt_read_at], books.index, fmt='.', tz=None, xdate=True, ydate=False)
    
plt.show();

ValueError: x and y must have same first dimension, but have shapes (2, 200) and (200,)

Error in callback <function install_repl_displayhook.<locals>.post_execute at 0x0000021C5FC73BF8> (for post_execute):


ValueError: view limit minimum -0.001 is less than 1 and is an invalid Matplotlib date value. This often happens if you pass a non-datetime value to an axis that has datetime units

ValueError: view limit minimum -0.001 is less than 1 and is an invalid Matplotlib date value. This often happens if you pass a non-datetime value to an axis that has datetime units

<Figure size 432x288 with 1 Axes>